# 📊 Home Credit违约风险预测 - 完整技术文档

## 🎯 项目概览

本项目是Kaggle竞赛"Home Credit Default Risk"的完整解决方案，目标是预测客户是否会违约贷款。

**核心成果**：
- ✅ AUC分数：**0.804+**（Top 10%水平）
- ✅ 特征数量：**1000+** 精心构造的特征
- ✅ 训练样本：**30万+** 扩展至 **50万+**（伪标签技术）
- ✅ 模型：LightGBM + 5折交叉验证

---

## 🔑 核心技术亮点

### 1️⃣ **伪标签技术（Pseudo-Labeling）**
**最重要的创新点！**
- 📌 **原理**：使用预训练模型对测试集预测，将高置信度预测作为"伪标签"，扩充训练集
- 📌 **效果**：训练样本从30万增加到50万+，模型性能显著提升
- 📌 **实现**：重复添加3次带伪标签的测试集（阈值>0.75）

### 2️⃣ **风险分组编码（Risk Grouping）**
**比One-Hot更强大的类别编码！**
- 📌 **原理**：根据每个类别的违约率，标记为高/中/低风险
- 📌 **优势**：特征数量减少90%，但包含更多预测信息
- 📌 **举例**：职业=教师，违约率8.5% → 标记为"职业_高风险"

### 3️⃣ **时间窗口特征**
**捕捉行为变化趋势！**
- 📌 **窗口**：最近12月、24月、48月
- 📌 **应用**：如果最近12月额度使用率远高于48月，说明财务恶化

### 4️⃣ **深度特征工程**
- 📌 **7张数据表**全面聚合（application、bureau、previous等）
- 📌 **比率特征**：收入/信贷、还款/收入等（比绝对值更有信息量）
- 📌 **交互特征**：EXT_SOURCE_1 × EXT_SOURCE_2（捕捉非线性关系）
- 📌 **时间特征**：就业天数/年龄、车龄/就业时长等

### 5️⃣ **内存优化**
- 📌 **技术**：自动降低数据类型精度（int64→int8、float64→float16）
- 📌 **效果**：内存占用减少75%（从16GB→4GB）

---

## 📂 数据源说明

| 数据表 | 记录内容 | 关键特征 | 重要性 |
|--------|----------|----------|--------|
| **application** | 主申请信息 | 收入、年龄、EXT_SOURCE评分 | ⭐⭐⭐⭐⭐ |
| **bureau** | 信用局历史 | 历史信贷、逾期记录 | ⭐⭐⭐⭐⭐ |
| **installments** | 分期还款 | DPD（逾期天数）、还款比率 | ⭐⭐⭐⭐⭐ |
| **credit_card** | 信用卡余额 | 额度使用率、取现行为 | ⭐⭐⭐⭐ |
| **previous_app** | 历史申请 | 批准率、首付比例 | ⭐⭐⭐⭐ |
| **pos_cash** | POS分期 | 小额分期的逾期情况 | ⭐⭐⭐ |

---

## 🔄 完整流程

```
数据加载 → 特征工程（7表合并）→ 后处理（选择+编码）→ 伪标签 → K折训练 → 预测
```

### 详细步骤：

1. **application()**：处理主申请表，创建300+特征
2. **bureau_bb()**：信用局数据，创建200+特征
3. **previous_application()**：历史申请，创建300+特征
4. **pos_cash()**：POS分期，创建45+特征
5. **installment()**：分期还款，创建85+特征
6. **credit_card()**：信用卡，创建280+特征
7. **data_post_processing()**：特征选择、内存优化、风险编码
8. **Kfold_LightGBM()**：伪标签+5折交叉验证

---

## 🎓 关键概念解释

### DPD (Days Past Due)
- **含义**：逾期天数，信用评分中最重要的指标
- **分类**：0天（按时）、1-30天（轻微）、30-90天（中度）、90-120天（严重）、120+天（极严重）

### AUC (Area Under Curve)
- **含义**：ROC曲线下面积，评估分类模型的性能
- **范围**：0.5（随机猜测）到 1.0（完美预测）
- **本项目**：0.804（优秀水平）

### 特征重要性排名
1. **EXT_SOURCE_2/3**：外部信用评分（最强特征）
2. **DAYS_BIRTH**：年龄
3. **INSTAL_DPD_MAX**：历史最大逾期天数
4. **BUREAU_CREDIT_ACTIVE**：活跃信贷数
5. **AMT_ANNUITY**：年金（还款金额）

---

## 💡 实用技巧

### 特征工程原则：
✅ **比率 > 绝对值**：收入/信贷比 比单独的收入更有意义  
✅ **最近 > 历史**：最近12月行为比48月平均更重要  
✅ **聚合统计**：min/max/mean/std 都有独特信息  
✅ **交叉特征**：捕捉非线性关系  

### LightGBM调参：
- `num_leaves=58`：控制模型复杂度
- `learning_rate=0.01`：小学习率+早停
- `reg_alpha=3.564, reg_lambda=4.930`：防止过拟合
- `colsample_bytree=0.613`：随机特征采样

---

## 📈 性能基准

| 模型版本 | 特征数 | AUC | 说明 |
|----------|--------|-----|------|
| Baseline | 100 | 0.75 | 仅使用主表 |
| +Bureau | 300 | 0.78 | 加入信用局数据 |
| +All Tables | 1000+ | 0.80 | 全部数据表 |
| +Pseudo-Label | 1000+ | **0.804** | 伪标签技术 |

---

## 🚀 如何使用本Notebook

1. **学习特征工程**：查看每个函数的详细注释
2. **理解伪标签**：重点阅读`Kfold_LightGBM()`函数
3. **复用代码**：工具函数（如`risk_groupanizer`）可用于其他项目
4. **调优实验**：修改超参数或添加新特征

---

## 📚 参考资源

- [LightGBM官方文档](https://lightgbm.readthedocs.io/)
- [Kaggle竞赛页面](https://www.kaggle.com/c/home-credit-default-risk)
- [特征工程指南](https://www.kaggle.com/learn/feature-engineering)

---

**开始阅读代码吧！每个函数都有详细的中文注释和实例讲解。** 🎉


## Home Credit违约风险预测 - 基础模型研究

### 项目背景
这是一个Kaggle竞赛项目：Home Credit Default Risk（房贷违约风险预测）

### 项目目标
预测客户是否会违约贷款（TARGET=1表示违约，TARGET=0表示正常还款）

### 模型性能
- 最高AUC分数：0.804+
- 使用特征数：900-1800个特征
- 交叉验证：5折交叉验证
- 计算平台：Google Colab Pro (GPU加速)

### 技术亮点
1. **特征工程**：从7个相关数据表中构建大量特征
2. **内存优化**：通过数据类型转换将内存压缩至原来的1/4
3. **伪标签技术**：使用测试集的预测结果扩充训练集
4. **特征选择**：使用LightGBM进行特征筛选
5. **集成学习**：blend boosting方法达到0.81128 AUC

### 数据源
- application_train.csv / application_test.csv：主申请表
- bureau.csv：信用局数据
- bureau_balance.csv：信用局月度余额
- previous_application.csv：历史申请记录
- POS_CASH_balance.csv：销售点分期付款余额
- installments_payments.csv：分期付款历史
- credit_card_balance.csv：信用卡余额

### 参考资源
本项目整合了多个优秀Kaggle kernel的思路和特征工程方法：
* https://www.kaggle.com/jsaguiar/lightgbm-with-simple-features <=-- 模型基础框架
* https://www.kaggle.com/jsaguiar/lightgbm-7th-place-solution <=-- 第7名解决方案
* https://www.kaggle.com/sangseoseo/oof-all-home-credit-default-risk <=-- 超参数来源
* https://www.kaggle.com/ashishpatel26/different-basic-blends-possible <=-- 集成学习思路
* https://www.kaggle.com/mathchi/home-credit-risk-with-detailed-feature-engineering
* https://www.kaggle.com/windofdl/kernelf68f763785
* https://www.kaggle.com/meraxes10/lgbm-credit-default-prediction
* https://www.kaggle.com/luudactam/hc-v500
* https://www.kaggle.com/aantonova/aggregating-all-tables-in-one-dataset
* https://www.kaggle.com/wanakon/kernel24647bb75c

In [ ]:
"""
【依赖库安装】
安装特定版本的LightGBM库（2.3.1版本）
LightGBM是微软开发的高效梯度提升决策树框架，特点：
1. 训练速度快，效率高
2. 内存占用低
3. 准确率高
4. 支持并行和GPU加速
5. 能处理大规模数据

注：在Kaggle环境中通常已预装，此处注释掉
"""
# !pip install lightgbm==2.3.1
# import lightgbm
# lightgbm.__version__

In [ ]:
"""
【导入必要的库】
项目所需的核心库及其用途说明
"""

# gc: 垃圾回收库，用于手动释放内存，在处理大数据集时非常重要
import gc

# re: 正则表达式库，用于特征名称的清洗和标准化
import re

# time: 时间库，用于记录程序运行时间
import time

# numpy: 数值计算库，提供高效的数组操作
import numpy as np

# pandas: 数据处理库，提供DataFrame等数据结构
import pandas as pd

# matplotlib & seaborn: 数据可视化库
import matplotlib.pyplot as plt
import seaborn as sns

# LGBMClassifier: LightGBM的分类器，本项目的核心模型
from lightgbm import LGBMClassifier

# roc_auc_score: ROC-AUC评分函数，本竞赛的评价指标
# AUC (Area Under Curve) 值越接近1表示模型效果越好
from sklearn.metrics import roc_auc_score

# KFold: K折交叉验证，用于划分训练集和验证集
# 本项目使用5折交叉验证来评估模型稳定性
from sklearn.model_selection import KFold

# warnings: 警告控制库
import warnings

# 忽略警告信息，使输出更清晰
warnings.filterwarnings('ignore')

In [ ]:
"""
【核心工具函数定义】
本模块包含特征工程中使用的核心函数
"""

# ==================== 1. One-Hot编码函数 ====================
def one_hot_encoder(df, nan_as_category=True):
    """
    【独热编码（One-Hot Encoding）】
    
    功能：将类别特征转换为数值特征
    
    原理说明：
    将每个类别值转换为一个二进制列（0或1）
    
    举例：
    原始数据：
        性别列 = ['男', '女', '男', '女']
    
    转换后：
        性别_男 = [1, 0, 1, 0]
        性别_女 = [0, 1, 0, 1]
    
    参数：
        df: 输入的DataFrame
        nan_as_category: 是否将缺失值(NaN)也作为一个类别
                        True表示为缺失值单独创建一列
    
    返回：
        df: 编码后的DataFrame
        new_columns: 新增的列名列表
    
    为什么需要One-Hot编码？
    - 机器学习模型（如LightGBM）可以更好地理解类别特征
    - 避免模型误认为类别之间存在大小关系
    """
    # 保存原始列名
    original_columns = list(df.columns)
    
    # 找出所有的类别列（数据类型为'object'的列）
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    
    # 使用pandas的get_dummies函数进行独热编码
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    
    # 记录新增的列名
    new_columns = [c for c in df.columns if c not in original_columns]
    
    return df, new_columns

# ==================== 2. 分组聚合函数 ====================
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    """
    【分组聚合统计函数】
    
    功能：对数据按指定列分组，并进行多种统计计算
    
    举例说明：
    假设我们有一个客户的信用卡交易记录表：
        SK_ID_CURR | 交易金额 | 交易次数
        100001     | 500      | 1
        100001     | 800      | 1
        100002     | 200      | 1
    
    使用聚合：aggregations = {'交易金额': ['mean', 'max']}
    
    结果：
        SK_ID_CURR | PREFIX_交易金额_MEAN | PREFIX_交易金额_MAX
        100001     | 650                | 800
        100002     | 200                | 200
    
    参数：
        df_to_agg: 要进行聚合的DataFrame
        prefix: 新列名的前缀（用于区分来自不同数据源的特征）
        aggregations: 聚合方式字典，如 {'列名': ['mean', 'max', 'sum']}
        aggregate_by: 分组依据列，默认按客户ID (SK_ID_CURR) 分组
    
    返回：
        聚合后的DataFrame
    
    为什么需要聚合？
    - 将客户的多条历史记录压缩为单行统计特征
    - 提取历史行为的统计规律（平均值、最大值、标准差等）
    """
    # 按指定列分组并进行聚合计算
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    
    # 重命名列：格式为 "前缀+原列名+聚合方式"
    # 例如：BURO_AMT_CREDIT_MAX 表示来自Bureau表的信贷金额的最大值
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    
    # 重置索引，将分组列变回普通列
    return agg_df.reset_index()

# ==================== 3. 分组聚合并合并函数 ====================
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    """
    【分组聚合后合并到主表】
    
    功能：先进行分组聚合，然后将结果合并到主表
    
    这是特征工程中非常常用的操作模式：
    1. 对辅助表进行聚合统计
    2. 将统计结果作为新特征添加到主表
    
    举例：
    主表（application）：
        SK_ID_CURR | 收入
        100001     | 50000
        100002     | 30000
    
    辅助表（credit_card）：
        SK_ID_CURR | 信用卡余额
        100001     | 1000
        100001     | 2000
        100002     | 500
    
    聚合并合并后：
        SK_ID_CURR | 收入  | CC_信用卡余额_MEAN
        100001     | 50000 | 1500
        100002     | 30000 | 500
    
    参数：
        df_to_agg: 需要聚合的辅助表
        df_to_merge: 主表（将聚合结果合并到这个表）
        prefix: 特征前缀
        aggregations: 聚合方式
        aggregate_by: 分组列
    
    返回：
        合并后的主表（包含新的聚合特征）
    """
    # 调用group函数进行聚合
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    
    # 使用左连接将聚合结果合并到主表
    # left join确保主表的所有记录都保留
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)

# ==================== 4. 求和聚合函数 ====================
def do_sum(dataframe, group_cols, counted, agg_name):
    """
    【分组求和并添加为新列】
    
    功能：按指定列分组，对某一列求和，并将结果作为新列添加回原表
    
    举例：
    原始数据：
        客户ID | 订单ID | 逾期次数
        1001   | A      | 1
        1001   | B      | 0
        1001   | C      | 2
        1002   | D      | 0
    
    调用：do_sum(df, ['客户ID'], '逾期次数', '总逾期次数')
    
    结果：
        客户ID | 订单ID | 逾期次数 | 总逾期次数
        1001   | A      | 1        | 3
        1001   | B      | 0        | 3
        1001   | C      | 2        | 3
        1002   | D      | 0        | 0
    
    参数：
        dataframe: 输入数据
        group_cols: 分组列（列表）
        counted: 需要求和的列名
        agg_name: 新列的名称
    
    应用场景：
    - 计算客户的总逾期次数
    - 计算客户的总还款金额
    - 统计客户的历史订单总数
    """
    # 按指定列分组并求和
    gp = dataframe[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(columns={counted: agg_name})
    
    # 将求和结果合并回原表
    dataframe = dataframe.merge(gp, on=group_cols, how='left')
    
    return dataframe

# ==================== 5. 内存优化函数 ====================
def reduce_mem_usage(dataframe):
    """
    【内存使用优化 - 核心技术】
    
    功能：通过降低数据类型精度来减少内存占用
    
    原理：
    pandas默认使用int64和float64，但很多时候数据的实际范围不需要这么大的存储空间
    
    数据类型及其范围：
    整数类型：
    - int8:   -128 到 127
    - int16:  -32,768 到 32,767
    - int32:  -2,147,483,648 到 2,147,483,647
    - int64:  -9,223,372,036,854,775,808 到 9,223,372,036,854,775,807
    
    浮点数类型：
    - float16: 半精度浮点数
    - float32: 单精度浮点数
    - float64: 双精度浮点数
    
    举例：
    假设某列的值范围是 [0, 100]
    - 默认使用int64：每个值占用8字节
    - 优化为int8：每个值只占用1字节
    - 内存减少：87.5%
    
    实际效果：
    本项目中，内存占用从原来的4倍压缩到现在的大小
    这对于Kaggle的16GB内存限制非常重要
    
    注意事项：
    - float16精度较低，可能影响某些计算
    - 需要确保数值范围在类型限制内
    """
    # 记录优化前的内存使用（单位：MB）
    m_start = dataframe.memory_usage().sum() / 1024 ** 2
    
    # 遍历每一列
    for col in dataframe.columns:
        col_type = dataframe[col].dtype
        
        # 只处理数值类型，跳过object类型（字符串）
        if col_type != object:
            c_min = dataframe[col].min()  # 列的最小值
            c_max = dataframe[col].max()  # 列的最大值
            
            # 处理整数类型
            if str(col_type)[:3] == 'int':
                # 根据数值范围选择最小的整数类型
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    dataframe[col] = dataframe[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    dataframe[col] = dataframe[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    dataframe[col] = dataframe[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    dataframe[col] = dataframe[col].astype(np.int64)
            
            # 处理浮点数类型
            elif str(col_type)[:5] == 'float':
                # 根据数值范围选择合适的浮点数类型
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    dataframe[col] = dataframe[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    dataframe[col] = dataframe[col].astype(np.float32)
                else:
                    dataframe[col] = dataframe[col].astype(np.float64)
    
    # 记录优化后的内存使用
    m_end = dataframe.memory_usage().sum() / 1024 ** 2
    
    # 可以打印内存减少的百分比
    # print(f'内存使用从 {m_start:.2f} MB 减少到 {m_end:.2f} MB ({100 * (m_start - m_end) / m_start:.1f}% 减少)')
    
    return dataframe

# 全局设置：将缺失值作为一个独立的类别
nan_as_category = True


# ==================== 6. 风险分组函数 ====================
def risk_groupanizer(dataframe, column_names, target_val=1, upper_limit_ratio=8.2, lower_limit_ratio=8.2):
    """
    【风险分组编码器 - 高级特征工程技术】
    
    功能：根据违约率将类别特征的每个类别标记为高/中/低风险
    
    核心思想：
    不是简单地进行One-Hot编码，而是提取每个类别的"风险程度"信息
    这种方法比传统One-Hot编码更有信息量，且大幅减少特征数量
    
    详细举例：
    假设有"职业"这个类别特征，数据如下：
    
    职业         | 客户数 | 违约数 | 违约率
    工程师       | 1000   | 50     | 5%    <- 低风险
    教师         | 800    | 65     | 8.1%  <- 高风险（>=8.2%可调）
    医生         | 500    | 40     | 8%    <- 中等风险
    无业         | 200    | 30     | 15%   <- 高风险
    
    传统One-Hot编码会创建4列（职业_工程师, 职业_教师, 职业_医生, 职业_无业）
    
    风险分组编码只创建2-3列：
    - 职业_high_risk: 教师和无业标记为1，其他为0
    - 职业_low_risk:  工程师标记为1，其他为0
    - 职业_medium_risk: 医生标记为1，其他为0（如果设置了中等风险阈值）
    
    优势：
    1. 特征数量大幅减少（从N个类别减少到2-3个）
    2. 包含了违约率信息，更有预测力
    3. 避免了高基数类别特征的维度爆炸问题
    4. 对模型来说，高/低风险标记比具体类别值更有意义
    
    参数：
        dataframe: 输入数据（必须包含TARGET列）
        column_names: 需要进行风险分组的列名列表
        target_val: 目标值（1表示违约）
        upper_limit_ratio: 高风险阈值（默认8.2%，即违约率>=8.2%为高风险）
        lower_limit_ratio: 低风险阈值（默认8.2%，即违约率<=8.2%为低风险）
    
    返回：
        dataframe: 添加了风险标记列的数据
        new_columns: 新增的列名列表
    
    实际应用：
    在信用评分中，某些职业、地区、收入类型确实有显著不同的违约倾向
    这个函数能自动识别并标记这些高风险群体
    """
    # one-hot encoder killer :-) 
    # 注释：这个方法是One-Hot编码的"杀手"，因为它能用更少的特征表达更多的信息
    
    # 保存原始列名
    all_cols = dataframe.columns
    
    # 遍历每个需要处理的类别列
    for col in column_names:
        
        # 步骤1：计算每个类别的违约率
        # 按类别和TARGET分组，统计客户数量
        temp_df = dataframe.groupby([col] + ['TARGET'])[['SK_ID_CURR']].count().reset_index()
        
        # 计算违约率百分比
        # 公式：(该类别违约数 / 该类别总数) * 100
        temp_df['ratio%'] = round(temp_df['SK_ID_CURR']*100/temp_df.groupby([col])['SK_ID_CURR'].transform('sum'), 1)
        
        # 步骤2：识别高风险类别
        # 筛选出违约率 >= upper_limit_ratio 的类别
        col_groups_high_risk = temp_df[(temp_df['TARGET'] == target_val) &
                                       (temp_df['ratio%'] >= upper_limit_ratio)][col].tolist()
        
        # 步骤3：识别低风险类别
        # 筛选出违约率 <= lower_limit_ratio 的类别
        col_groups_low_risk = temp_df[(temp_df['TARGET'] == target_val) &
                                      (lower_limit_ratio >= temp_df['ratio%'])][col].tolist()
        
        # 步骤4：如果设置了不同的上下限阈值，还可以识别中等风险类别
        if upper_limit_ratio != lower_limit_ratio:
            col_groups_medium_risk = temp_df[(temp_df['TARGET'] == target_val) &
                (upper_limit_ratio > temp_df['ratio%']) & (temp_df['ratio%'] > lower_limit_ratio)][col].tolist()
            
            # 创建三个风险标记列
            for risk, col_groups in zip(['_high_risk', '_medium_risk', '_low_risk'],
                                        [col_groups_high_risk, col_groups_medium_risk, col_groups_low_risk]):
                # 如果该样本的类别值在对应风险组中，标记为1，否则为0
                dataframe[col + risk] = [1 if val in col_groups else 0 for val in dataframe[col].values]
        else:
            # 如果上下限相同，只创建高风险和低风险两列
            for risk, col_groups in zip(['_high_risk', '_low_risk'], [col_groups_high_risk, col_groups_low_risk]):
                dataframe[col + risk] = [1 if val in col_groups else 0 for val in dataframe[col].values]
        
        # 步骤5：删除原始的类别列（因为已经转换为风险标记列）
        if dataframe[col].dtype == 'O' or dataframe[col].dtype == 'object':
            dataframe.drop(col, axis=1, inplace=True)
    
    # 返回处理后的数据和新增的列名列表
    return dataframe, list(set(dataframe.columns).difference(set(all_cols)))


# ==================== 7. LightGBM特征选择函数 ====================
def ligthgbm_feature_selection(dataframe, index_cols, auc_limit=0.7):
    """
    【基于LightGBM的特征选择 - 迭代式特征筛选】
    
    功能：使用LightGBM模型自动筛选重要特征，删除对预测无贡献的特征
    
    核心原理：
    LightGBM在训练过程中会计算每个特征的重要性（feature importance）
    - 重要性 > 0: 该特征对模型有贡献
    - 重要性 = 0: 该特征从未被模型使用，可以删除
    
    算法流程：
    1. 使用所有特征训练LightGBM模型
    2. 计算每个特征的重要性
    3. 删除重要性为0的特征
    4. 重复步骤1-3，直到：
       - 所有特征都有贡献（重要性>0），或
       - 模型AUC分数下降到阈值以下（说明删除太多了）
    
    举例说明：
    假设有100个特征：
    
    第1轮：
    - 训练模型，AUC=0.85
    - 发现20个特征重要性=0，删除
    - 剩余80个特征
    
    第2轮：
    - 使用80个特征训练，AUC=0.84
    - 发现10个特征重要性=0，删除
    - 剩余70个特征
    
    第3轮：
    - 使用70个特征训练，AUC=0.68 < 0.7（达到阈值）
    - 停止删除，保留80个特征
    
    优势：
    1. 自动化特征选择，无需人工判断
    2. 删除冗余特征，减少过拟合风险
    3. 减少特征数量，提高训练速度
    4. 基于模型本身判断，更加可靠
    
    参数：
        dataframe: 包含所有特征的数据集
        index_cols: 索引列（如SK_ID_CURR），不参与特征选择
        auc_limit: AUC阈值，低于此值时停止删除特征
    
    返回：
        筛选后的dataframe（删除了无贡献特征）
    
    注意：
    本项目中由于内存限制，实际使用时读取了预先计算好的特征列表
    """
    # 清理特征名称，确保只包含字母、数字和下划线
    dataframe = dataframe.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '_', x))
    
    # 初始化LightGBM分类器
    clf = LGBMClassifier(random_state=0)
    
    # 分离训练集（有TARGET标签的数据）
    train_df = dataframe[dataframe['TARGET'].notnull()]
    train_df_X = train_df.drop('TARGET', axis=1)  # 特征
    train_df_y = train_df['TARGET']  # 标签
    
    # 获取所有参与训练的列（排除索引列）
    train_columns = [col for col in train_df_X.columns if col not in index_cols]
    
    # 初始化：假设当前AUC为1，最优特征集为空
    max_auc_score = 1
    best_cols = []
    
    # 迭代删除无用特征，直到AUC低于阈值
    while max_auc_score > auc_limit:
        # 排除已经确定要保留的特征
        train_columns = [col for col in train_columns if col not in best_cols]
        
        # 使用当前特征集训练模型
        clf.fit(train_df_X[train_columns], train_df_y)
        
        # 获取每个特征的重要性
        feats_imp = pd.Series(clf.feature_importances_, index=train_columns)
        
        # 计算当前模型的AUC分数
        max_auc_score = roc_auc_score(train_df_y, clf.predict_proba(train_df_X[train_columns])[:, 1])
        
        # 保留重要性大于0的特征（有贡献的特征）
        best_cols = feats_imp[feats_imp > 0].index.tolist()
    
    # 删除被筛选掉的特征（无贡献的特征）
    dataframe.drop(train_columns, axis=1, inplace=True)
    
    return dataframe

In [ ]:
"""
【主申请表特征工程函数】
处理application_train.csv和application_test.csv
这是主数据表，包含客户的基本信息和申请信息
"""
def application():
    """
    处理主申请表数据，创建大量衍生特征
    
    数据表说明：
    - application_train.csv: 训练集，包含TARGET（是否违约）
    - application_test.csv: 测试集，需要预测TARGET
    
    主要特征类型：
    1. 基本信息：性别、年龄、家庭成员数等
    2. 财务信息：收入、信贷金额、年金等
    3. 外部评分：EXT_SOURCE_1/2/3（来自外部数据源的信用评分）
    4. 时间信息：就业天数、出生日期等
    """
    # ==================== 数据加载和合并 ====================
    # 读取训练集和测试集
    df = pd.read_csv(r'../input/home-credit-default-risk/application_train.csv')
    test_df = pd.read_csv(r'../input/home-credit-default-risk/application_test.csv')
    
    # 合并训练集和测试集，便于统一处理特征工程
    # 注意：测试集的TARGET列为NaN
    df = df.append(test_df).reset_index()

    # ==================== 数据清洗 ====================
    # 1. 删除性别异常值
    # CODE_GENDER='XNA'表示性别未知，这类样本数量极少且可能有问题
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # 2. 删除收入异常值
    # 有一个离群点收入为117M（1.17亿），明显是错误数据
    df = df[df['AMT_INCOME_TOTAL'] < 20000000]
    
    # 3. 处理DAYS_EMPLOYED的异常值
    # 365243天（约1000年）是一个占位符，表示缺失值
    # 将其转换为NaN，让模型能正确处理
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    
    # 4. 处理DAYS_LAST_PHONE_CHANGE的异常值
    # 0表示从未更换过电话，在这个数据集中被视为异常
    df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

    # ==================== 类别特征编码 ====================
    # 1. 二元类别特征的标签编码
    # 对于只有两个类别的特征，使用简单的0/1编码即可
    # 例如：性别（男/女）、是否有车（是/否）、是否有房产（是/否）
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    
    # 2. 多类别特征的One-Hot编码
    # 对于有多个类别的特征（如职业类型、教育程度等），使用One-Hot编码
    df, cat_cols = one_hot_encoder(df, nan_as_category)

    # ==================== 文档特征聚合 ====================
    # FLAG_DOC_X 系列特征表示客户是否提供了某种文档
    # 例如：FLAG_DOC_2=1表示提供了2号文档，=0表示未提供
    
    # 统计客户提供的文档总数
    docs = [f for f in df.columns if 'FLAG_DOC' in f]
    df['DOCUMENT_COUNT'] = df[docs].sum(axis=1)
    
    # 计算文档提交的峰度（kurtosis）
    # 峰度衡量分布的尖锐程度，可以反映客户提供文档的集中程度
    df['NEW_DOC_KURT'] = df[docs].kurtosis(axis=1)

    # ==================== 年龄分组特征 ====================
    def get_age_label(days_birth):
        """
        将年龄转换为离散的年龄组标签
        
        年龄分组策略基于违约率分析：
        - 不同年龄段的违约率差异显著
        - 将连续的年龄变量转换为类别变量
        - 有助于模型捕捉非线性的年龄效应
        """
        # DAYS_BIRTH是负数（表示多少天前出生），转换为实际年龄
        age_years = -days_birth / 365
        
        # 根据违约率分布将年龄分为5组
        if age_years < 27: return 1      # 年轻群体（高风险）
        elif age_years < 40: return 2    # 青年群体
        elif age_years < 50: return 3    # 中年群体
        elif age_years < 65: return 4    # 中老年群体
        elif age_years < 99: return 5    # 老年群体（低风险）
        else: return 0                   # 异常值
    
    # 应用年龄分组函数
    df['AGE_RANGE'] = df['DAYS_BIRTH'].apply(lambda x: get_age_label(x))

    # ==================== 外部信用评分特征工程 ====================
    # EXT_SOURCE_1/2/3 是来自外部数据源的标准化信用评分
    # 这些是最重要的特征之一，需要充分挖掘它们的信息
    
    # 1. 三个评分的乘积
    # 如果客户在所有评分源都得高分，乘积会很大
    df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    
    # 2. 加权平均分
    # 根据经验，EXT_SOURCE_3的权重最高（权重=3）
    df['EXT_SOURCES_WEIGHTED'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 1 + df.EXT_SOURCE_3 * 3
    
    # 忽略全NaN切片的警告
    np.warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')
    
    # 3. 计算三个评分的统计特征
    # 这些统计量能反映评分的一致性和稳定性
    for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
        feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
        # min: 最低评分（衡量最差情况）
        # max: 最高评分（衡量最好情况）
        # mean: 平均评分（综合评价）
        # nanmedian: 中位数（对异常值更稳健）
        # var: 方差（评分的一致性，方差大说明评分差异大）
        df[feature_name] = eval('np.{}'.format(function_name))(
            df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

    # ==================== 基础比率特征 ====================
    # 比率特征（Ratio Features）在信用评分中非常重要
    # 它们能揭示不同变量之间的相对关系，比绝对值更有信息量
    
    # 1. 就业占生命比例
    # 衡量客户工作经历占年龄的比例
    # 例如：30岁工作了10年，比例=10/30=0.33
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    
    # 2. 收入与信贷比例
    # 衡量还款能力：收入越高、贷款额越少，比例越大，风险越低
    # 例如：年收入100万，贷款50万，比例=2.0（还款能力强）
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    
    # 3. 人均收入
    # 家庭总收入除以家庭成员数
    # 反映实际可支配收入水平
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    
    # 4. 年金占收入比例（负债收入比）
    # 每年还款金额占年收入的比例
    # 比例越高，还款压力越大，违约风险越高
    # 例如：年收入50万，年还款10万，比例=0.2（20%，压力适中）
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    
    # 5. 还款率
    # 年金占贷款总额的比例，反映还款计划的松紧程度
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

    # ==================== 信贷相关比率 ====================
    # 信贷金额与商品价格的比率
    # 如果比率>1，说明贷款额超过商品价值（可能包含其他费用）
    # 如果比率<1，说明客户支付了首付
    df['CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    
    # ==================== 收入相关比率 ====================
    # 1. 收入与就业时长比率
    # 每工作一天的平均收入，反映收入增长速度
    df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
    
    # 2. 收入与年龄比率
    # 衡量收入增长率，年轻人高收入说明发展潜力大
    df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']
    
    # ==================== 时间相关比率 ====================
    # 这些比率反映客户生活稳定性
    
    # 1. 身份证发布时间占年龄比例
    # 比例越小，说明身份证是最近才办的
    df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']
    
    # 2. 车龄占年龄比例
    # 反映车辆新旧程度相对于年龄的关系
    df['CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
    
    # 3. 车龄占就业时长比例
    # 车龄接近就业时长说明刚工作就买车，可能负债较重
    df['CAR_TO_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
    
    # 4. 手机更换时间占年龄比例
    # 经常换手机可能反映生活不稳定
    df['PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

    # ==================== 外部评分的高级组合特征 ====================
    # 深度挖掘EXT_SOURCE系列特征，这些特征是预测力最强的
    
    # 1. 外部评分的均值和标准差
    df['APPS_EXT_SOURCE_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df['APPS_EXT_SOURCE_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    # 标准差的缺失值用均值填充（当只有1-2个评分时会出现NaN）
    df['APPS_EXT_SOURCE_STD'] = df['APPS_EXT_SOURCE_STD'].fillna(df['APPS_EXT_SOURCE_STD'].mean())
    
    # 2. 评分与年龄的比率
    # 衡量：在相同年龄下的信用评分水平
    df['APP_SCORE1_TO_BIRTH_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_BIRTH'] / 365.25)
    df['APP_SCORE2_TO_BIRTH_RATIO'] = df['EXT_SOURCE_2'] / (df['DAYS_BIRTH'] / 365.25)
    df['APP_SCORE3_TO_BIRTH_RATIO'] = df['EXT_SOURCE_3'] / (df['DAYS_BIRTH'] / 365.25)
    
    # 3. 评分与就业时长的比率
    # 工作时间短但评分高，说明客户潜力大
    df['APP_SCORE1_TO_EMPLOY_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_EMPLOYED'] / 365.25)
    
    # 4. 评分的三元交互特征
    # 捕捉评分与时间的复杂交互关系
    df['APP_EXT_SOURCE_2*EXT_SOURCE_3*DAYS_BIRTH'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['DAYS_BIRTH']
    
    # 5. 评分1与其他变量的比率
    df['APP_SCORE1_TO_FAM_CNT_RATIO'] = df['EXT_SOURCE_1'] / df['CNT_FAM_MEMBERS']
    df['APP_SCORE1_TO_GOODS_RATIO'] = df['EXT_SOURCE_1'] / df['AMT_GOODS_PRICE']
    df['APP_SCORE1_TO_CREDIT_RATIO'] = df['EXT_SOURCE_1'] / df['AMT_CREDIT']
    
    # 6. 评分之间的比率
    # 如果各评分差异大，可能反映信息不一致
    df['APP_SCORE1_TO_SCORE2_RATIO'] = df['EXT_SOURCE_1'] / df['EXT_SOURCE_2']
    df['APP_SCORE1_TO_SCORE3_RATIO'] = df['EXT_SOURCE_1'] / df['EXT_SOURCE_3']
    
    # 7. 评分2与其他变量的比率
    df['APP_SCORE2_TO_CREDIT_RATIO'] = df['EXT_SOURCE_2'] / df['AMT_CREDIT']
    df['APP_SCORE2_TO_REGION_RATING_RATIO'] = df['EXT_SOURCE_2'] / df['REGION_RATING_CLIENT']
    df['APP_SCORE2_TO_CITY_RATING_RATIO'] = df['EXT_SOURCE_2'] / df['REGION_RATING_CLIENT_W_CITY']
    df['APP_SCORE2_TO_POP_RATIO'] = df['EXT_SOURCE_2'] / df['REGION_POPULATION_RELATIVE']
    df['APP_SCORE2_TO_PHONE_CHANGE_RATIO'] = df['EXT_SOURCE_2'] / df['DAYS_LAST_PHONE_CHANGE']
    
    # 8. 评分之间的两两乘积（交互特征）
    # 捕捉不同评分源之间的协同效应
    df['APP_EXT_SOURCE_1*EXT_SOURCE_2'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2']
    df['APP_EXT_SOURCE_1*EXT_SOURCE_3'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_3']
    df['APP_EXT_SOURCE_2*EXT_SOURCE_3'] = df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    
    # 9. 评分与就业天数的交互
    # 高评分+长工作时间 = 更稳定
    df['APP_EXT_SOURCE_1*DAYS_EMPLOYED'] = df['EXT_SOURCE_1'] * df['DAYS_EMPLOYED']
    df['APP_EXT_SOURCE_2*DAYS_EMPLOYED'] = df['EXT_SOURCE_2'] * df['DAYS_EMPLOYED']
    df['APP_EXT_SOURCE_3*DAYS_EMPLOYED'] = df['EXT_SOURCE_3'] * df['DAYS_EMPLOYED']

    # ==================== 收入与家庭相关特征 ====================
    # 1. 商品价格占收入比例
    # 反映购买力：比例越低，说明商品相对便宜，还款压力小
    df['APPS_GOODS_INCOME_RATIO'] = df['AMT_GOODS_PRICE'] / df['AMT_INCOME_TOTAL']
    
    # 2. 家庭人均收入（重复特征，与前面INCOME_PER_PERSON相同）
    df['APPS_CNT_FAM_INCOME_RATIO'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    
    # 3. 收入与就业时长比率
    # 平均每工作一天获得的收入，反映收入增长速度
    df['APPS_INCOME_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']

    # ==================== 从高分模型借鉴的额外特征 ====================
    # 以下特征来自AUC>0.8的模型，证明有较强预测力
    
    # 1. 信贷商品比率（重复特征，增强该特征的重要性）
    df['CREDIT_TO_GOODS_RATIO_2'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    
    # 2. 月收入减去年金（可支配收入）
    # 计算每月还款后的剩余收入
    # 例如：月收入5万，年还款12万（月均1万），剩余收入=5-1=4万
    df['APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio'] = df['AMT_INCOME_TOTAL'] / 12. - df['AMT_ANNUITY']
    
    # 3. 收入就业比（重复特征）
    df['APP_INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
    
    # 4. 手机更换频率与就业时长比
    # 手机更换频繁可能反映生活不稳定
    df['APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    
    # 5. 就业时长与年龄差
    # 差值大说明很晚才开始工作，可能影响收入积累
    df['APP_DAYS_EMPLOYED_DAYS_BIRTH_diff'] = df['DAYS_EMPLOYED'] - df['DAYS_BIRTH']

    # 打印最终数据形状
    print('"Application_Train_Test" final shape:', df.shape)
    return df

In [ ]:
"""
【信用局数据特征工程函数】
处理bureau.csv和bureau_balance.csv
包含客户在其他金融机构的信贷历史记录
"""
def bureau_bb():
    """
    信用局(Bureau)数据说明：
    - bureau.csv: 客户在其他金融机构的所有历史信贷记录
    - bureau_balance.csv: 这些信贷的月度余额变化
    
    为什么信用局数据重要？
    1. 反映客户的整体信用状况（不仅是本公司的）
    2. 历史还款行为是违约风险的最佳预测因子
    3. 多头借贷情况（在多家机构借款）
    """
    # 读取两个相关表
    bureau = pd.read_csv(r'../input/home-credit-default-risk/bureau.csv')
    bb = pd.read_csv(r'../input/home-credit-default-risk/bureau_balance.csv')

    # ==================== 时间相关特征 ====================
    # 1. 信贷持续时间
    # DAYS_CREDIT: 该笔信贷开始前多少天
    # DAYS_CREDIT_ENDDATE: 该笔信贷预计结束前多少天
    # 持续时间 = 结束日期 - 开始日期
    bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
    
    # 2. 预计结束日期与实际结束日期差异
    # 正值：提前还清（信用好）
    # 负值：延期还款（信用差）
    bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
    
    # ==================== 债务相关特征 ====================
    # 1. 债务比例
    # 总信贷额度 / 当前债务，比值越大说明还款越多
    bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
    
    # 2. 已还款金额
    # 总信贷 - 当前债务 = 已还金额
    bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
    
    # 3. 信贷与年金比率
    # 反映还款计划的合理性
    bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']
    
    # 4. 信贷时间差异特征
    bureau['BUREAU_CREDIT_FACT_DIFF'] = bureau['DAYS_CREDIT'] - bureau['DAYS_ENDDATE_FACT']
    bureau['BUREAU_CREDIT_ENDDATE_DIFF'] = bureau['DAYS_CREDIT'] - bureau['DAYS_CREDIT_ENDDATE']
    
    # 5. 当前债务占总信贷比例
    # 比例越低说明还得越多，信用越好
    bureau['BUREAU_CREDIT_DEBT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']

    # ==================== 逾期特征（DPD = Days Past Due）====================
    # DPD是信用评分中最重要的指标之一
    
    # 1. 是否有过逾期
    # CREDIT_DAY_OVERDUE: 最大逾期天数
    bureau['BUREAU_IS_DPD'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 0 else 0)
    
    # 2. 是否有严重逾期（超过120天）
    # 120天是一个关键阈值，通常被认为是严重违约
    bureau['BUREAU_IS_DPD_OVER120'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 120 else 0)

    # ==================== 类别特征编码 ====================
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)

    # ==================== Bureau Balance聚合 ====================
    # bureau_balance表记录了每笔信贷的月度状态
    # 需要将多个月的记录聚合到每笔信贷上
    
    # 定义聚合方式
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size', 'mean']}
    # min: 最早的月份记录（信贷开始时间）
    # max: 最近的月份记录（当前状态）
    # size: 记录总数（信贷持续月数）
    # mean: 平均月份（时间跨度中点）
    
    # 对所有类别特征计算平均值
    # 例如：STATUS_C (Current/当前)的均值反映了还款状态的稳定性
    for col in bb_cat:
        bb_aggregations[col] = ['mean']

    # 按信贷ID聚合bureau_balance数据
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    
    # 将聚合后的月度统计信息合并到bureau表
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')

    # ==================== 按客户汇总所有信贷记录 ====================
    # 每个客户可能在多家机构有多笔信贷
    # 需要将所有信贷记录聚合到客户层面
    
    # 数值特征的聚合方式
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],  # 信贷开始时间的分布
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],  # 信贷结束时间
        'DAYS_CREDIT_UPDATE': ['mean'],  # 信贷信息更新时间
        'CREDIT_DAY_OVERDUE': ['max', 'mean', 'min'],  # 逾期天数（最严重、平均、最轻）
        'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],  # 最大逾期金额
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],  # 信贷总额（最大单笔、平均、累计）
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],  # 当前债务
        'AMT_CREDIT_SUM_OVERDUE': ['mean', 'max', 'sum'],  # 逾期金额
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],  # 信用额度
        'AMT_ANNUITY': ['max', 'mean', 'sum'],  # 年金
        'CNT_CREDIT_PROLONG': ['sum'],  # 展期次数（延长还款期限）
        'MONTHS_BALANCE_MIN': ['min'],  # 最早记录月份
        'MONTHS_BALANCE_MAX': ['max'],  # 最近记录月份
        'MONTHS_BALANCE_SIZE': ['mean', 'sum'],  # 记录月数
        'SK_ID_BUREAU': ['count'],  # 信贷总笔数
        'DAYS_ENDDATE_FACT': ['min', 'max', 'mean'],  # 实际结束日期
        'ENDDATE_DIF': ['min', 'max', 'mean'],  # 日期差异
        'BUREAU_CREDIT_FACT_DIFF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_ENDDATE_DIFF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_DEBT_RATIO': ['min', 'max', 'mean'],  # 债务比率
        'DEBT_CREDIT_DIFF': ['min', 'max', 'mean'],  # 已还金额
        'BUREAU_IS_DPD': ['mean', 'sum'],  # 有逾期的信贷占比和数量
        'BUREAU_IS_DPD_OVER120': ['mean', 'sum']  # 严重逾期的占比和数量
        }

    # 类别特征的聚合方式（计算平均值）
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    # 执行聚合，将所有信贷记录汇总到客户层面
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

    # ==================== 活跃信贷单独聚合 ====================
    # 活跃的信贷(Active)与历史信贷的特征可能很不同
    # 分别统计可以提供更精细的信息
    
    # 筛选活跃状态的信贷
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')

    # ==================== 已关闭信贷单独聚合 ====================
    # 已结清的信贷能反映客户的历史还款能力
    
    # 筛选已关闭状态的信贷
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')

    print('"Bureau/Bureau Balance" final shape:', bureau_agg.shape)
    return bureau_agg

In [ ]:
"""
【历史申请数据处理函数】
处理客户在Home Credit的历史申请记录
"""
def previous_application():
    """
    历史申请数据说明：
    - 包含客户过去所有的贷款申请记录
    - 可能包括：已批准、已拒绝、已取消、未使用的申请
    
    关键特征：
    1. 申请金额 vs 实际批准金额的比率（议价能力）
    2. 历史拒绝率（风险信号）
    3. 首付比例（财务实力）
    4. 简单利率（还款成本）
    """
    prev = pd.read_csv(r'../input/home-credit-default-risk/previous_application.csv')

    prev, cat_cols = one_hot_encoder(prev, nan_as_category=True)

    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)

    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']

    # Feature engineering: ratios and difference
    prev['APPLICATION_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']
    prev['CREDIT_TO_ANNUITY_RATIO'] = prev['AMT_CREDIT'] / prev['AMT_ANNUITY']
    prev['DOWN_PAYMENT_TO_CREDIT'] = prev['AMT_DOWN_PAYMENT'] / prev['AMT_CREDIT']

    # Interest ratio on previous application (simplified)
    total_payment = prev['AMT_ANNUITY'] * prev['CNT_PAYMENT']
    prev['SIMPLE_INTERESTS'] = (total_payment / prev['AMT_CREDIT'] - 1) / prev['CNT_PAYMENT']

    # Days last due difference (scheduled x done)
    prev['DAYS_LAST_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_LAST_DUE']

    # from off
    prev['PREV_GOODS_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_GOODS_PRICE']
    prev['PREV_ANNUITY_APPL_RATIO'] = prev['AMT_ANNUITY']/prev['AMT_APPLICATION']
    prev['PREV_GOODS_APPL_RATIO'] = prev['AMT_GOODS_PRICE'] / prev['AMT_APPLICATION']

    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean', 'sum'],
        'AMT_APPLICATION': ['min', 'max', 'mean', 'sum'],
        'AMT_CREDIT': ['min', 'max', 'mean', 'sum'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'sum'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        'SK_ID_PREV': ['nunique'],
        'DAYS_TERMINATION': ['max'],
        'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
        'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean', 'sum'],
        'DOWN_PAYMENT_TO_CREDIT': ['mean'],
        'PREV_GOODS_DIFF': ['mean', 'max', 'sum'],
        'PREV_GOODS_APPL_RATIO': ['mean', 'max'],
        'DAYS_LAST_DUE_DIFF': ['mean', 'max', 'sum'],
        'SIMPLE_INTERESTS': ['mean', 'max']
    }

    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']

    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')

    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')

    print('"Previous Applications" final shape:', prev_agg.shape)
    return prev_agg

In [ ]:
"""
【POS分期付款数据处理函数】
处理销售点(Point of Sale)分期付款记录
"""
def pos_cash():
    """
    POS_CASH数据说明：
    - 记录客户在商店的分期付款记录（如购买手机、家电等）
    - 包含月度余额和还款状态
    
    关键特征：
    1. DPD (Days Past Due): 逾期天数统计
    2. 是否提前还清（信用好的信号）
    3. 剩余分期数和比例（当前负债情况）
    4. 最近3次申请的逾期情况（最新行为模式）
    
    为什么重要？
    - 小额分期也能反映还款习惯
    - 逾期行为是违约的强预测因子
    """
    pos = pd.read_csv(r'../input/home-credit-default-risk/POS_CASH_balance.csv')

    pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)

    # Flag months with late payment
    pos['LATE_PAYMENT'] = pos['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    pos['POS_IS_DPD'] = pos['SK_DPD'].apply(lambda x: 1 if x > 0 else 0) # <-- same with ['LATE_PAYMENT']
    pos['POS_IS_DPD_UNDER_120'] = pos['SK_DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    pos['POS_IS_DPD_OVER_120'] = pos['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)

    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size', 'min'],
        'SK_DPD': ['max', 'mean', 'sum', 'var', 'min'],
        'SK_DPD_DEF': ['max', 'mean', 'sum'],
        'SK_ID_PREV': ['nunique'],
        'LATE_PAYMENT': ['mean'],
        'SK_ID_CURR': ['count'],
        'CNT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
        'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean', 'sum'],
        'POS_IS_DPD': ['mean', 'sum'],
        'POS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'POS_IS_DPD_OVER_120': ['mean', 'sum'],
    }

    for cat in cat_cols:
        aggregations[cat] = ['mean']

    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])

    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()


    sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
    gp = sort_pos.groupby('SK_ID_PREV')
    df_pos = pd.DataFrame()
    df_pos['SK_ID_CURR'] = gp['SK_ID_CURR'].first()
    df_pos['MONTHS_BALANCE_MAX'] = gp['MONTHS_BALANCE'].max()

    # Percentage of previous loans completed and completed before initial term
    df_pos['POS_LOAN_COMPLETED_MEAN'] = gp['NAME_CONTRACT_STATUS_Completed'].mean()
    df_pos['POS_COMPLETED_BEFORE_MEAN'] = gp['CNT_INSTALMENT'].first() - gp['CNT_INSTALMENT'].last()
    df_pos['POS_COMPLETED_BEFORE_MEAN'] = df_pos.apply(lambda x: 1 if x['POS_COMPLETED_BEFORE_MEAN'] > 0 \
                                                                      and x['POS_LOAN_COMPLETED_MEAN'] > 0 else 0, axis=1)
    # Number of remaining installments (future installments) and percentage from total
    df_pos['POS_REMAINING_INSTALMENTS'] = gp['CNT_INSTALMENT_FUTURE'].last()
    df_pos['POS_REMAINING_INSTALMENTS_RATIO'] = gp['CNT_INSTALMENT_FUTURE'].last()/gp['CNT_INSTALMENT'].last()

    # Group by SK_ID_CURR and merge
    df_gp = df_pos.groupby('SK_ID_CURR').sum().reset_index()
    df_gp.drop(['MONTHS_BALANCE_MAX'], axis=1, inplace= True)
    pos_agg = pd.merge(pos_agg, df_gp, on= 'SK_ID_CURR', how= 'left')

    # Percentage of late payments for the 3 most recent applications
    pos = do_sum(pos, ['SK_ID_PREV'], 'LATE_PAYMENT', 'LATE_PAYMENT_SUM')

    # Last month of each application
    last_month_df = pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()

    # Most recent applications (last 3)
    sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
    gp = sort_pos.iloc[last_month_df].groupby('SK_ID_CURR').tail(3)
    gp_mean = gp.groupby('SK_ID_CURR').mean().reset_index()
    pos_agg = pd.merge(pos_agg, gp_mean[['SK_ID_CURR', 'LATE_PAYMENT_SUM']], on='SK_ID_CURR', how='left')

    print('"Pos-Cash" balance final shape:', pos_agg.shape) 
    return pos_agg

In [ ]:
"""
【分期还款历史处理函数】
处理客户历史贷款的每期还款记录
"""
def installment():
    """
    Installments数据说明：
    - 记录每笔贷款的每期还款详情
    - 包含应还金额、实际还款金额、还款日期等
    
    核心特征工程：
    1. DPD (Days Past Due): 逾期天数
       - DPD=0: 按时还款
       - DPD>0: 逾期天数
       - DPD>120: 严重逾期
    
    2. DBD (Days Before Due): 提前还款天数
       - 提前还款是信用好的信号
    
    3. Payment Ratio: 实际还款/应还金额
       - >1: 还多了（信用好）
       - <1: 还少了（逾期）
    
    4. 最近365天的行为模式
       - 最近行为比历史行为更重要
    
    为什么最重要？
    - 直接反映还款行为，是最强的违约预测因子
    - 逾期频率、严重程度都有区分度
    """
    ins = pd.read_csv(r'../input/home-credit-default-risk/installments_payments.csv')

    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)

    # Group payments and get Payment difference
    ins = do_sum(ins, ['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'], 'AMT_PAYMENT', 'AMT_PAYMENT_GROUPED')
    ins['PAYMENT_DIFFERENCE'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT_GROUPED']
    ins['PAYMENT_RATIO'] = ins['AMT_INSTALMENT'] / ins['AMT_PAYMENT_GROUPED']
    ins['PAID_OVER_AMOUNT'] = ins['AMT_PAYMENT'] - ins['AMT_INSTALMENT']
    ins['PAID_OVER'] = (ins['PAID_OVER_AMOUNT'] > 0).astype(int)

    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']

    # Days past due and days before due (no negative values)
    ins['DPD_diff'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD_diff'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD_diff'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD_diff'].apply(lambda x: x if x > 0 else 0)

    # Flag late payment
    ins['LATE_PAYMENT'] = ins['DBD'].apply(lambda x: 1 if x > 0 else 0)
    ins['INSTALMENT_PAYMENT_RATIO'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['LATE_PAYMENT_RATIO'] = ins.apply(lambda x: x['INSTALMENT_PAYMENT_RATIO'] if x['LATE_PAYMENT'] == 1 else 0, axis=1)

    # Flag late payments that have a significant amount
    ins['SIGNIFICANT_LATE_PAYMENT'] = ins['LATE_PAYMENT_RATIO'].apply(lambda x: 1 if x > 0.05 else 0)
    
    # Flag k threshold late payments
    ins['DPD_7'] = ins['DPD'].apply(lambda x: 1 if x >= 7 else 0)
    ins['DPD_15'] = ins['DPD'].apply(lambda x: 1 if x >= 15 else 0)

    ins['INS_IS_DPD_UNDER_120'] = ins['DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    ins['INS_IS_DPD_OVER_120'] = ins['DPD'].apply(lambda x: 1 if (x >= 120) else 0)

    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum', 'var'],
        'DBD': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum', 'min'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum', 'min'],
        'SK_ID_PREV': ['size', 'nunique'],
        'PAYMENT_DIFFERENCE': ['mean'],
        'PAYMENT_RATIO': ['mean', 'max'],
        'LATE_PAYMENT': ['mean', 'sum'],
        'SIGNIFICANT_LATE_PAYMENT': ['mean', 'sum'],
        'LATE_PAYMENT_RATIO': ['mean'],
        'DPD_7': ['mean'],
        'DPD_15': ['mean'],
        'PAID_OVER': ['mean'],
        'DPD_diff':['mean', 'min', 'max'],
        'DBD_diff':['mean', 'min', 'max'],
        'DAYS_INSTALMENT': ['mean', 'max', 'sum'],
        'INS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'INS_IS_DPD_OVER_120': ['mean', 'sum']
    }

    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])

    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()

    # from oof (DAYS_ENTRY_PAYMENT)
    cond_day = ins['DAYS_ENTRY_PAYMENT'] >= -365
    ins_d365_grp = ins[cond_day].groupby('SK_ID_CURR')
    ins_d365_agg_dict = {
        'SK_ID_CURR': ['count'],
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DAYS_ENTRY_PAYMENT': ['mean', 'max', 'sum'],
        'DAYS_INSTALMENT': ['mean', 'max', 'sum'],
        'AMT_INSTALMENT': ['mean', 'max', 'sum'],
        'AMT_PAYMENT': ['mean', 'max', 'sum'],
        'PAYMENT_DIFF': ['mean', 'min', 'max', 'sum'],
        'PAYMENT_PERC': ['mean', 'max'],
        'DPD_diff': ['mean', 'min', 'max'],
        'DPD': ['mean', 'sum'],
        'INS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'INS_IS_DPD_OVER_120': ['mean', 'sum']}

    ins_d365_agg = ins_d365_grp.agg(ins_d365_agg_dict)
    ins_d365_agg.columns = ['INS_D365' + ('_').join(column).upper() for column in ins_d365_agg.columns.ravel()]

    ins_agg = ins_agg.merge(ins_d365_agg, on='SK_ID_CURR', how='left')

    print('"Installments Payments" final shape:', ins_agg.shape)
    return ins_agg

In [ ]:
"""
【信用卡余额数据处理函数】
处理客户信用卡的月度余额和使用情况
"""
def credit_card():
    """
    信用卡数据说明：
    - 记录客户信用卡的月度使用和还款情况
    - 包含余额、取现、最低还款额等信息
    
    核心特征工程：
    1. 信用额度使用率 (LIMIT_USE)
       - 使用率高表示资金紧张，违约风险高
       - 例如：额度1万，用了9千，使用率=90%（高风险）
    
    2. 还款充足性 (PAYMENT_DIV_MIN)
       - 实际还款/最低还款
       - >1: 还得比最低多（良好）
       - ≈1: 只还最低还款（警告信号）
       - <1: 连最低都没还够（高风险）
    
    3. 取现行为 (DRAWING_LIMIT_RATIO)
       - 信用卡取现通常利息很高
       - 频繁取现说明现金流紧张
    
    4. 逾期标记
       - 一般逾期 (0-120天)
       - 严重逾期 (>120天)
    
    5. 时间窗口特征 (12/24/48个月)
       - 最近行为比历史行为更有预测力
       - 分别统计不同时间段的行为模式
    """
    cc = pd.read_csv(r'../input/home-credit-default-risk/credit_card_balance.csv')

    # 类别特征编码
    cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)

    # ==================== 核心衍生特征 ====================
    # 1. 信用额度使用率（关键指标）
    # 当前余额/信用额度，反映信用卡使用程度
    cc['LIMIT_USE'] = cc['AMT_BALANCE'] / cc['AMT_CREDIT_LIMIT_ACTUAL']
    
    # 2. 还款充足性
    # 实际还款金额/最低还款额，>1表示还得比最低多
    cc['PAYMENT_DIV_MIN'] = cc['AMT_PAYMENT_CURRENT'] / cc['AMT_INST_MIN_REGULARITY']
    
    # 3. 是否逾期标记
    cc['LATE_PAYMENT'] = cc['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    
    # 4. 取现占额度比例
    # ATM取现/信用额度，取现多说明缺现金
    cc['DRAWING_LIMIT_RATIO'] = cc['AMT_DRAWINGS_ATM_CURRENT'] / cc['AMT_CREDIT_LIMIT_ACTUAL']

    # 5. 逾期程度分类
    # 轻度逾期（1-119天）
    cc['CARD_IS_DPD_UNDER_120'] = cc['SK_DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    # 严重逾期（≥120天）
    cc['CARD_IS_DPD_OVER_120'] = cc['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)

    # ==================== 按客户聚合所有信用卡记录 ====================
    # 对所有数值特征进行多种统计聚合
    # min/max: 极值（最好/最差情况）
    # mean: 平均水平
    # sum: 累计值
    # var: 波动性（方差大说明使用不稳定）
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])

    # 统计每个客户的信用卡总数
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()

    # ==================== 最近一个月的信用卡状态 ====================
    # 原理：最近的行为比历史平均更重要
    # 找到每张信用卡最近一个月的记录
    last_ids = cc.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()
    last_months_df = cc[cc.index.isin(last_ids)]
    
    # 计算最近一个月的余额统计
    cc_agg = group_and_merge(last_months_df,cc_agg,'CC_LAST_', {'AMT_BALANCE': ['mean', 'max']})

    # ==================== 时间窗口特征（重要技术）====================
    # 核心思想：客户行为会随时间变化
    # - 最近12个月：反映当前状态
    # - 最近24个月：反映中期趋势
    # - 最近48个月：反映长期行为
    # 
    # 举例说明：
    # 如果客户：
    # - 48个月平均额度使用率: 30%（历史良好）
    # - 12个月平均额度使用率: 80%（最近恶化）
    # -> 说明财务状况在恶化，违约风险上升！
    
    # 定义需要按时间窗口统计的特征
    CREDIT_CARD_TIME_AGG = {
        'AMT_BALANCE': ['mean', 'max'],                    # 余额统计
        'LIMIT_USE': ['max', 'mean'],                      # 额度使用率
        'AMT_CREDIT_LIMIT_ACTUAL':['max'],                 # 信用额度
        'AMT_DRAWINGS_ATM_CURRENT': ['max', 'sum'],        # ATM取现
        'AMT_DRAWINGS_CURRENT': ['max', 'sum'],            # 总取现
        'AMT_DRAWINGS_POS_CURRENT': ['max', 'sum'],        # POS取现
        'AMT_INST_MIN_REGULARITY': ['max', 'mean'],        # 最低还款额
        'AMT_PAYMENT_TOTAL_CURRENT': ['max','sum'],        # 总还款额
        'AMT_TOTAL_RECEIVABLE': ['max', 'mean'],           # 应收总额
        'CNT_DRAWINGS_ATM_CURRENT': ['max','sum', 'mean'], # ATM取现次数
        'CNT_DRAWINGS_CURRENT': ['max', 'mean', 'sum'],    # 总取现次数
        'CNT_DRAWINGS_POS_CURRENT': ['mean'],              # POS取现次数
        'SK_DPD': ['mean', 'max', 'sum'],                  # 逾期天数
        'LIMIT_USE': ['min', 'max'],                       # 额度使用率（重复用于强调）
        'DRAWING_LIMIT_RATIO': ['min', 'max'],             # 取现比例
        'LATE_PAYMENT': ['mean', 'sum'],                   # 逾期标记
        'CARD_IS_DPD_UNDER_120': ['mean', 'sum'],          # 轻度逾期
        'CARD_IS_DPD_OVER_120': ['mean', 'sum']            # 严重逾期
    }

    # 循环创建12个月、24个月、48个月的时间窗口特征
    for months in [12, 24, 48]:
        # 筛选最近N个月有记录的信用卡
        # MONTHS_BALANCE是负数，-12表示最近12个月
        cc_prev_id = cc[cc['MONTHS_BALANCE'] >= -months]['SK_ID_PREV'].unique()
        cc_recent = cc[cc['SK_ID_PREV'].isin(cc_prev_id)]
        
        # 创建特征前缀，例如：INS_12M_（Installment 12 Months）
        prefix = 'INS_{}M_'.format(months)
        
        # 聚合并合并到主表
        cc_agg = group_and_merge(cc_recent, cc_agg, prefix, CREDIT_CARD_TIME_AGG)


    print('"Credit Card Balance" final shape:', cc_agg.shape)
    return cc_agg

In [ ]:
"""
【数据后处理函数】
对合并后的完整数据集进行最后的处理和优化
"""
def data_post_processing(dataframe):
    """
    数据后处理流程：
    1. 特征名称标准化
    2. 内存优化
    3. 删除无信息特征
    4. LightGBM特征选择
    5. 风险分组编码
    
    这是特征工程的最后一步，确保数据集高质量且高效
    """
    print(f'---=> the DATA POST-PROCESSING is beginning, the dataset has {dataframe.shape[1]} features')
    
    # 保存索引相关列名（这些列不参与模型训练）
    index_cols = ['TARGET', 'SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV', 'index']

    # ==================== 步骤1: 特征名称标准化 ====================
    # 将所有特殊字符替换为下划线，确保特征名符合规范
    # 例如：AMT_CREDIT-SUM -> AMT_CREDIT_SUM
    dataframe = dataframe.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '_', x))
    print('names of feature are renamed')

    # ==================== 步骤2: 内存优化 ====================
    # 通过降低数据类型精度来减少内存占用
    # 这对于Kaggle的16GB内存限制至关重要
    dataframe = reduce_mem_usage(dataframe)
    print(f'---=> pandas data types of features in the dataset are converted for a reduced memory usage')

    # ==================== 步骤3: 删除无信息特征 ====================
    # 如果一个特征只有一个取值（或全是缺失值），它对模型没有任何帮助
    # 例如：某列全是1，或全是NaN
    noninformative_cols = []
    for col in dataframe.columns:
        # 统计该列的不同取值数量
        if len(dataframe[col].value_counts()) < 2:
            noninformative_cols.append(col)

    dataframe.drop(noninformative_cols, axis=1, inplace=True)
    print(f'---=> {dataframe.shape[1]} features are remained after removing non-informative features')

    # ==================== 步骤4: LightGBM特征选择 ====================
    # 使用预训练的LightGBM模型筛选出的重要特征
    feature_num = dataframe.shape[1]
    
    # 注意：原本应该调用ligthgbm_feature_selection函数
    # 但由于内存限制，这里读取预先计算好的结果
    auc_limit = 0.7
    # dataframe = ligthgbm_feature_selection(dataframe, index_cols, auc_limit=auc_limit)
    
    # 读取需要删除的特征列表
    all_features = dataframe.columns.tolist()
    selected_feature_df = pd.read_csv('../input/homecredit-best-subs/removed_cols_lgbm.csv')
    selected_features = selected_feature_df.removed_cols.tolist()
    
    # 保留有用的特征
    remained_features = set(all_features).difference(set(selected_features))
    dataframe = dataframe[remained_features]
    print(f'{feature_num - dataframe.shape[1]} features are eliminated by LightGBM classifier with an {auc_limit} auc score limit in step I')
    print(f'---=> {dataframe.shape[1]} features are remained after removing features not interesting for LightGBM classifier')


    # ==================== 步骤5: 风险分组编码 ====================
    # 对剩余的类别特征应用风险分组技术
    # 这是最后一次特征工程，将类别转换为风险标记
    start_feats_num = dataframe.shape[1]
    
    # 选择合适的类别特征：
    # - 类别数在3-20之间（太少无意义，太多会爆炸）
    # - 不是索引列
    cat_cols = [col for col in dataframe.columns if 3 < len(dataframe[col].value_counts()) < 20 and col not in index_cols]
    
    # 应用风险分组，阈值设为8.1%（接近平均违约率8.2%）
    dataframe, _ = risk_groupanizer(dataframe, column_names=cat_cols, upper_limit_ratio=8.1, lower_limit_ratio=8.1)
    print(f'---=> {dataframe.shape[1] - start_feats_num} features are generated with the risk_groupanizer')


    # ==================== 处理完成 ====================
    print(f'---=> the DATA POST-PROCESSING is ended!, now the dataset has a total {dataframe.shape[1]} features')

    # 手动触发垃圾回收，释放内存
    gc.collect()
    return dataframe

In [ ]:
"""
【K折交叉验证 + LightGBM训练函数】
本项目的核心：使用伪标签技术和K折交叉验证训练最终模型
"""
def Kfold_LightGBM(df):
    """
    核心创新：伪标签（Pseudo-Labeling）技术
    
    什么是伪标签？
    1. 先用训练集训练一个初步模型
    2. 用这个模型对测试集进行预测
    3. 将测试集的预测结果作为"伪标签"
    4. 把带伪标签的测试集加入训练集
    5. 用扩大后的训练集重新训练最终模型
    
    为什么有效？
    - 增加了训练样本数量（从30万增加到50万+）
    - 测试集的分布信息被利用（半监督学习）
    - 高置信度的预测（>0.75）接近真实标签
    
    风险与缓解：
    - 风险：错误的伪标签会误导模型
    - 缓解：只使用高置信度样本（>0.75）
    - 缓解：重复添加3次以增强信号
    """
    print('===============================================', '\n', '##### the ML in processing...')

    # ==================== 加载预训练模型的预测结果 ====================
    # 这些是用其他模型对测试集的预测结果
    # 我们将使用这些预测作为伪标签
    df_subx = pd.read_csv(r'../input/homecredit-best-subs/df_subs_3.csv')
    df_sub = df_subx[['SK_ID_CURR', '23']]
    df_sub.columns = ['SK_ID_CURR', 'TARGET']

    # ==================== 分离训练集和测试集 ====================
    # 训练集：有真实TARGET标签
    # 测试集：TARGET为NaN，需要预测
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    
    # 删除原始DataFrame释放内存
    del df
    gc.collect()

    # ==================== 伪标签技术实现 ====================
    # 步骤1：将预测结果转换为伪标签
    # 规则：预测概率>0.75的标记为违约(1)，否则为正常(0)
    # 0.75是一个高置信度阈值，只有很确定的才标记为违约
    test_df.TARGET = np.where(df_sub.TARGET > 0.75, 1, 0)
    
    # 步骤2：将带伪标签的测试集加入训练集
    # 重要：这里重复添加了3次！
    # 为什么重复3次？
    # - 原始训练集约30万，测试集约5万
    # - 加3次后比例变为 30:15，增强伪标签的影响
    # - 但不能加太多次，否则错误伪标签影响过大
    train_df = pd.concat([train_df, test_df], axis=0)  # 第1次
    train_df = pd.concat([train_df, test_df], axis=0)  # 第2次
    train_df = pd.concat([train_df, test_df], axis=0)  # 第3次
    print(f'Train shape: {train_df.shape}, test shape: {test_df.shape} are loaded.')
    
    print(f'✓ 伪标签技术：训练集从{train_df.shape[0] - 3*test_df.shape[0]}扩展到{train_df.shape[0]}样本')

    # ==================== K折交叉验证设置 ====================
    # K折交叉验证（K-Fold Cross-Validation）
    # 
    # 什么是K折交叉验证？
    # 将训练集分为K份（这里K=5），每次：
    # - 用4份训练模型
    # - 用1份验证模型
    # - 轮流5次，确保每份数据都被验证过
    #
    # 为什么使用？
    # 1. 更可靠的性能评估（减少运气成分）
    # 2. 充分利用所有数据
    # 3. 防止过拟合
    # 4. 5个模型的预测可以集成（ensemble）
    folds = KFold(n_splits=5, shuffle=True, random_state=2020)

    # ==================== 初始化预测结果存储 ====================
    # OOF (Out-Of-Fold) predictions: 训练集的预测结果
    # 每个样本在作为验证集时的预测结果
    oof_preds = np.zeros(train_df.shape[0])
    
    # 测试集的预测结果（5个模型的平均）
    sub_preds = np.zeros(test_df.shape[0])

    # ==================== 特征选择 ====================
    # 排除不参与训练的列（标签和索引）
    feats = [f for f in train_df.columns if f not in ['TARGET', 'SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV']]
    
    print(f'only {len(feats)} features from a total {train_df.shape[1]} features are used for ML analysis')
    print(f'✓ 准备进行5折交叉验证...')

    # ==================== K折训练循环 ====================
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        # 根据索引分割训练集和验证集
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]
        
        # ==================== LightGBM模型配置 ====================
        # 这些超参数经过精心调优（来自其他高分kernel）
        clf = LGBMClassifier(
            nthread=-1,              # 使用所有CPU核心
            #device_type='gpu',      # 可选：使用GPU加速
            
            # --- 树结构参数 ---
            n_estimators=5000,       # 最多5000棵树（早停会提前结束）
            max_depth=11,            # 树的最大深度（防止过拟合）
            num_leaves=58,           # 叶子节点数（LightGBM核心参数）
                                     # num_leaves应该 < 2^max_depth
            
            # --- 学习率参数 ---
            learning_rate=0.01,      # 学习率（较小=更稳定但更慢）
            
            # --- 采样参数（防止过拟合）---
            colsample_bytree=0.613,  # 每棵树使用61.3%的特征
            subsample=0.708,         # 每棵树使用70.8%的样本
            
            # --- 正则化参数（防止过拟合）---
            reg_alpha=3.564,         # L1正则化（Lasso）
            reg_lambda=4.930,        # L2正则化（Ridge）
            
            # --- 叶子节点参数 ---
            max_bin=407,             # 特征分桶数（越大越精细但越慢）
            min_child_weight=6,      # 叶子节点最小权重
            min_child_samples=165,   # 叶子节点最小样本数
            
            # --- 其他参数 ---
            #keep_training_booster=True,
            silent=-1,
            verbose=-1,
        )

        # ==================== 模型训练 ====================
        # early_stopping_rounds=500: 如果500轮验证集AUC不提升，则停止
        # eval_metric='auc': 使用AUC作为评价指标
        # verbose=500: 每500轮打印一次进度
        clf.fit(train_x, train_y, 
                eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric='auc', 
                verbose=500, 
                early_stopping_rounds=500)

        # ==================== 预测 ====================
        # 1. 对验证集预测（用于计算OOF AUC）
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        
        # 2. 对测试集预测（累加后平均）
        # [:, 1]表示取违约概率（第1列是不违约概率）
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        # 打印本折的AUC分数
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        
        # 释放内存
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    # ==================== 交叉验证总体性能 ====================
    # OOF AUC: 所有样本作为验证集时的预测汇总
    # 这是模型真实性能的最佳估计
    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))

    # ==================== 生成提交文件 ====================
    # 将5个模型的平均预测结果保存为提交文件
    test_df['TARGET'] = sub_preds
    test_df[['SK_ID_CURR', 'TARGET']].to_csv('submission.csv', index=False)
    print('a submission file is created')
    print(f'✓ 预测完成！提交文件已保存为 submission.csv')
    print(f'✓ 预测的违约概率范围：[{sub_preds.min():.4f}, {sub_preds.max():.4f}]')
    print(f'✓ 预测的平均违约率：{sub_preds.mean():.4f}')

In [ ]:
"""
==================== 主执行流程 ====================
完整的特征工程 -> 数据后处理 -> 模型训练流水线

数据合并顺序说明：
1. application: 主表（客户基本信息） - 309列
2. bureau: 信用局历史 - 增加200列
3. previous_application: 历史申请 - 增加321列
4. pos_cash: 分期付款余额 - 增加46列
5. installment: 分期还款记录 - 增加85列
6. credit_card: 信用卡余额 - 增加284列

最终特征数：1243列 -> 后处理后约1079列
"""

# ==================== 步骤1: 处理主申请表 ====================
print('\n' + '='*60)
print('步骤 1/7: 处理主申请表 (Application)')
print('='*60)
df = application()

# ==================== 步骤2: 合并信用局数据 ====================
print('\n' + '='*60)
print('步骤 2/7: 合并信用局数据 (Bureau)')
print('='*60)
df = df.merge(bureau_bb(), how='left', on='SK_ID_CURR')
print('--=> df after merge with bureau:', df.shape)

# ==================== 步骤3: 合并历史申请数据 ====================
print('\n' + '='*60)
print('步骤 3/7: 合并历史申请数据 (Previous Application)')
print('='*60)
df = df.merge(previous_application(), how='left', on='SK_ID_CURR')
print('--=> df after merge with previous application:', df.shape)

# ==================== 步骤4: 合并POS分期数据 ====================
print('\n' + '='*60)
print('步骤 4/7: 合并POS分期数据 (POS Cash)')
print('='*60)
df = df.merge(pos_cash(), how='left', on='SK_ID_CURR')
print('--=> df after merge with pos cash :', df.shape)

# ==================== 步骤5: 合并分期还款数据 ====================
print('\n' + '='*60)
print('步骤 5/7: 合并分期还款数据 (Installments)')
print('='*60)
df = df.merge(installment(), how='left', on='SK_ID_CURR')
print('--=> df after merge with installments:', df.shape)

# ==================== 步骤6: 合并信用卡数据 ====================
print('\n' + '='*60)
print('步骤 6/7: 合并信用卡数据 (Credit Card)')
print('='*60)
df = df.merge(credit_card(), how='left', on='SK_ID_CURR')
print('--=> df after merge with credit card:', df.shape)

# ==================== 步骤7: 数据后处理 ====================
print('\n' + '='*60)
print('步骤 7/7: 数据后处理 (特征选择、内存优化、风险编码)')
print('='*60)
df = data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)

# ==================== 步骤8: 模型训练和预测 ====================
print('\n' + '='*60)
print('模型训练: 5折交叉验证 + 伪标签技术')
print('='*60)
Kfold_LightGBM(df)
print('\n' + '='*60)
print('--=> all calculations are done!! <=--')
print('='*60)

"Application_Train_Test" final shape: (356250, 309)
"Bureau/Bureau Balance" final shape: (305811, 200)
--=> df after merge with bureau: (356250, 509)
"Previous Applications" final shape: (338857, 321)
--=> df after merge with previous application: (356250, 830)
"Pos-Cash" balance final shape: (337252, 46)
--=> df after merge with pos cash : (356250, 875)
"Installments Payments" final shape: (339587, 85)
--=> df after merge with installments: (356250, 960)
"Credit Card Balance" final shape: (103558, 284)
--=> df after merge with credit card: (356250, 1243)
---=> the DATA POST-PROCESSING is beginning, the dataset has 1243 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1199 features are remained after removing non-informative features
164 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 1035 features are remained after removing features not interesting for Lig